### Train Phrase

In [1]:
import numpy as np
import cv2
import os
import glob
import time
import sklearn
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from skimage.feature import hog
import extract_features
import slide_windows
from scipy.ndimage.measurements import label
%matplotlib inline

In [2]:
from collections import deque
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

#### Setting up (load data and subset it)

In [3]:
vehicle_folds = os.listdir('./data/vehicles')
non_vehicle_folds = os.listdir('./data/non-vehicles')
non_vehicles = []
vehicles = []

for folder in vehicle_folds:
    vehicles.extend(glob.glob('./data/vehicles/' + folder + '/*.png'))

for folder in non_vehicle_folds:
    non_vehicles.extend(glob.glob('./data/non-vehicles/' + folder + '/*.png'))

example = cv2.imread(vehicles[0])
print('Shape: ', example.shape)
print('pixel value: ', np.min(example), " - ", np.max(example))

Shape:  (64, 64, 3)
pixel value:  33  -  192


In [4]:
np.random.seed(123)
num_subset = 3000
if num_subset == 'ALL':
    car_subset = vehicles
    notcar_subset = non_vehicles
else:
    car_subset = np.array(vehicles)[np.random.choice(len(vehicles), num_subset)]
    notcar_subset = np.array(non_vehicles)[np.random.choice(len(non_vehicles), num_subset)]

print('Number of vehicle image: {0}, Number of non-vehicle image: {1}'.
      format(len(vehicles), len(non_vehicles)))
print('Number of subset vehicle image: {0}, Number of subset non-vehicle image: {1}'.
      format(len(car_subset), len(notcar_subset)))

Number of vehicle image: 8792, Number of non-vehicle image: 8968
Number of subset vehicle image: 3000, Number of subset non-vehicle image: 3000


#### Training model

In [5]:
# hog feature extration
hog_params = {'cspace': 'YCrCb',
             'orient': 9,
             'pix_per_cell': 8,
             'cell_per_block': 2,
             'hog_channel': 'ALL'}

In [6]:
t = time.time()
car_features = extract_features.extract_f(car_subset, 
                         hog_params)
notcar_features = extract_features.extract_f(notcar_subset,  
                         hog_params)
X = np.vstack((car_features, notcar_features))
y = np.hstack((np.ones(len(car_subset)), np.zeros(len(notcar_subset))))
print('Extracting features took: {0:.2f} seconds to extract {1} features'.
      format(time.time() - t, X.shape[1]))

Extracting features took: 35.48 seconds to extract 5292 features


In [7]:
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [8]:
rand_state = np.random.randint(0, 100)
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.3, random_state=rand_state)

In [9]:
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)

In [10]:
def train_model(train, target, val, model, use_pca, use_bagging,
               bag_params=None):
    if use_pca:
        pca = PCA(n_components = 500)
        pca.fit(X_train)
        print(np.sum(pca.explained_variance_ratio_))
        train = pca.transform(train)
        val = pca.transform(val)
    if use_bagging:
        model = BaggingClassifier(base_estimator=model,
                                      n_estimators=bag_params['n_estimators'],
                                      max_features=bag_params['max_features'],
                                      max_samples=bag_params['max_samples'])
    model.fit(train, target)
    if use_pca:
        return model, pca, train, val
    else:
        return model, train, val

In [11]:
use_pca = True
use_bagging = True
bag_params = {'n_estimators': 10,
             'max_features': 0.7,
             'max_samples': 0.5}
nn = MLPClassifier(hidden_layer_sizes=(50, 10), activation='relu', alpha = 1e-1, solver='adam', early_stopping = True, verbose=False)

if use_pca:
    model, pca, train, val = train_model(X_train, y_train, X_val, nn, use_pca, use_bagging, bag_params)
else:
    model, train, val = train_model(X_train, y_train, X_val, nn, use_pca, use_bagging, bag_params)

0.9603067050186835


In [12]:
print('Training acc: {0:.4f}'.format(model.score(train, y_train)))
print('Validation acc: {0:.4f}'.format(model.score(val, y_val)))

Training acc: 0.9583
Validation acc: 0.9294


### Test Phrase

In [13]:
def draw_heatmap(heatmap, found):
    for box in found:
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1
    return heatmap

def draw_heatmap_deque(heatmap, d):
    for found in d:
        for box in found:
            heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1
    return heatmap


def threshold(heatmap, thres):
    heatmap[heatmap < thres] = 0
    return heatmap

def draw_labeled_bboxes(image, labels):
    # Iterate through all detected cars
    image = image.copy()
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        # Draw the box on the image
        cv2.rectangle(image, bbox[0], bbox[1], (0,0,255), 6)
    # Return the image
    return image

In [14]:
def find_cars(img, y_start_stops, x_start_stops, scale, model, scaler, use_pca=True, pca=None,
              cells_per_step = 2, hog_params = None):
    
    ystart, ystop = y_start_stops
    xstart, xstop = x_start_stops
    
    img_tosearch = img[ystart:ystop,xstart:xstop,:]
    ctrans_tosearch = img_tosearch
    if hog_params['cspace'] != 'RGB':
                exec("%s = %s" % ('ctrans_tosearch',
                                  'cv2.cvtColor(img_tosearch, cv2.COLOR_RGB2' + hog_params['cspace'] + ')')) 
    if scale != 1:
        imshape = ctrans_tosearch.shape
        ctrans_tosearch = cv2.resize(ctrans_tosearch, (np.int(imshape[1]/scale), np.int(imshape[0]/scale)))
        
    ch1 = ctrans_tosearch[:,:,0]
    ch2 = ctrans_tosearch[:,:,1]
    ch3 = ctrans_tosearch[:,:,2]

    # Define blocks and steps as above
    nxblocks = (ch1.shape[1] // hog_params['pix_per_cell']) - hog_params['cell_per_block'] + 1
    nyblocks = (ch1.shape[0] // hog_params['pix_per_cell']) - hog_params['cell_per_block'] + 1 
    nfeat_per_block = hog_params['orient']*hog_params['cell_per_block']**2
    
    # 64 was the orginal sampling rate, with 8 cells and 8 pix per cell
    window = 64
    nblocks_per_window = (window // hog_params['pix_per_cell']) - hog_params['cell_per_block'] + 1
    cells_per_step = 2  # Instead of overlap, define how many cells to step
    nxsteps = (nxblocks - nblocks_per_window) // cells_per_step + 1
    nysteps = (nyblocks - nblocks_per_window) // cells_per_step + 1
    
    # Compute individual channel HOG features for the entire image
    hog1 = extract_features.get_hog_features(ch1,
                            hog_params['orient'],
                            hog_params['pix_per_cell'],
                            hog_params['cell_per_block'],
                            feature_vec=False)
    hog2 = extract_features.get_hog_features(ch2,
                            hog_params['orient'],
                            hog_params['pix_per_cell'],
                            hog_params['cell_per_block'],
                            feature_vec=False)
    hog3 = extract_features.get_hog_features(ch3,
                            hog_params['orient'],
                            hog_params['pix_per_cell'],
                            hog_params['cell_per_block'],
                            feature_vec=False)
    
    found = []
    for xb in range(nxsteps):
        for yb in range(nysteps):
            ypos = yb*cells_per_step
            xpos = xb*cells_per_step
            # Extract HOG for this patch
            hog_feat1 = hog1[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_feat2 = hog2[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_feat3 = hog3[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_features = np.hstack((hog_feat1, hog_feat2, hog_feat3))

            xleft = xpos*hog_params['pix_per_cell']
            ytop = ypos*hog_params['pix_per_cell']
            
            # Scale features and make a prediction
            test_features = scaler.transform(hog_features).reshape(1, -1)    

            if use_pca:
                test_features = pca.transform(test_features)
            
            pred = 1 if model.predict_proba(test_features)[:,1] > prob else 0
            
            if pred == 1:
                xbox_left = np.int(xleft*scale)
                ytop_draw = np.int(ytop*scale)
                win_draw = np.int(window*scale)
                box = ((xbox_left+xstart, ytop_draw+ystart),(xbox_left+win_draw+xstart,ytop_draw+win_draw+ystart)) 
                found.append(box)
    return found

In [15]:
def process_image(image):
    
    found = []
    
    for scale in scales:
        found.extend(find_cars(image, y_start_stops, x_start_stops, scale,
                      model, scaler, use_pca, pca, cells_per_step, hog_params))
    #print(len(found))
    
    d.append(found)
    
    heatmap = np.zeros((image.shape[0:2]))
    heatmap = draw_heatmap_deque(heatmap, d)
    headmap = threshold(heatmap, threshold_pixel)

    labels = label(heatmap)

    window_img = draw_labeled_bboxes(image, labels)

    #window_img = slide_windows.draw_boxes(image, found, color=(0, 0, 255), thick=6) 
    
    return window_img

In [21]:
test_clip = VideoFileClip("./test_video.mp4")

In [22]:
scales = [1.25, 1.5]
y_start_stops = (380, 700)
x_start_stops = (780, 1280)
cells_per_step = 2
prob = 0.8
threshold_pixel = 4
d = deque(maxlen=5)

out_clip = test_clip.fl_image(process_image)
output = './out_test_clip.mp4'
%time out_clip.write_videofile(output, audio=False)

[MoviePy] >>>> Building video ./out_test_clip.mp4
[MoviePy] Writing video ./out_test_clip.mp4


 97%|███████████████████████████████████████████████████████████████████████████████▉  | 38/39 [03:40<00:05,  5.80s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./out_test_clip.mp4 

Wall time: 3min 41s


In [23]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output))

In [24]:
clip25 = VideoFileClip("./project_video_lane_finding.mp4")

In [27]:
'''
scales = [1.25, 1.5]
y_start_stops = (380, 700)
x_start_stops = (750, 1280)
cells_per_step = 2
prob = 0.8
threshold_pixel = 4
d = deque(maxlen=5)

out_clip_25_multi = clip25.fl_image(process_image)
output_25 = './out_clip_25_multi.mp4'
%time out_clip_25_multi.write_videofile(output_25, audio=False)
'''

[MoviePy] >>>> Building video ./out_clip_25_multi.mp4
[MoviePy] Writing video ./out_clip_25_multi.mp4


100%|███████████████████████████████████████████████████████████████████████████▉| 1260/1261 [2:11:20<00:06,  6.21s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./out_clip_25_multi.mp4 

Wall time: 2h 11min 20s


In [28]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output_25))